In [1]:
import os
import requests
from dotenv import load_dotenv
from IPython.display import display, Markdown

# Load environment variables from .env
load_dotenv()

def generate_and_format_response(
    provider,
    model,
    messages,
    temperature=0.7,
    max_tokens=1500,
    top_p=0.9,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    min_tokens=None,
    stream=False,
    stop=None,
    random_seed=None,
    response_format=None,
    tools=None,
    tool_choice="auto",
    safe_prompt=False
):
    """
    Generates and formats a response from the specified LLM provider (OpenAI, Anthropic, or Mistral).
    
    Parameters:
    - provider (str): The LLM provider to use ("openai", "anthropic", or "mistral").
    - model (str): The model to use for text generation.
    - messages (list): A list of messages in the format [{"role": "user", "content": "Your prompt here"}].
    - temperature (float, optional): Controls randomness in the output (default: 0.7 for Anthropic & Mistral, 1.0 for OpenAI).
    - max_tokens (int, optional): Max tokens to generate (default: 1500).
    - top_p (float, optional): Nucleus sampling parameter for controlling diversity (default: 0.9).
    - frequency_penalty (float, optional): Penalizes new tokens based on frequency (OpenAI only).
    - presence_penalty (float, optional): Penalizes new tokens based on presence (OpenAI only).
    - Additional options are specific to certain providers.
    
    Returns:
    - formatted_text (str): The formatted assistant response in Markdown.
    """
    
    api_key = None
    url = None
    headers = {"Content-Type": "application/json"}

    # Determine API key, endpoint, and headers based on provider
    if provider.lower() == "openai":
        api_key = os.getenv("OPENAI_API_KEY")
        url = "https://api.openai.com/v1/chat/completions"
        headers["Authorization"] = f"Bearer {api_key}"
        payload = {
            "model": model,
            "messages": messages,
            "temperature": temperature,
            "max_tokens": max_tokens,
            "top_p": top_p,
            "frequency_penalty": frequency_penalty,
            "presence_penalty": presence_penalty
        }

    elif provider.lower() == "anthropic":
        api_key = os.getenv("ANTHROPIC_API_KEY")
        url = "https://api.anthropic.com/v1/messages"
        headers["x-api-key"] = api_key
        headers["anthropic-version"] = "2023-06-01"
        payload = {
            "model": model,
            "temperature": temperature,
            "max_tokens": max_tokens,
            "top_p": top_p,
            "messages": messages
        }

    elif provider.lower() == "mistral":
        api_key = os.getenv("MISTRAL_API_KEY")
        url = "https://api.mistral.ai/v1/chat/completions"
        headers["Authorization"] = f"Bearer {api_key}"
        payload = {
            "model": model,
            "messages": messages,
            "temperature": temperature,
            "top_p": top_p,
            "stream": stream,
            "tool_choice": tool_choice,
            "safe_prompt": safe_prompt
        }
        # Add optional parameters for Mistral
        if max_tokens is not None:
            payload["max_tokens"] = max_tokens
        if min_tokens is not None:
            payload["min_tokens"] = min_tokens
        if stop is not None:
            payload["stop"] = stop
        if random_seed is not None:
            payload["random_seed"] = random_seed
        if response_format is not None:
            payload["response_format"] = response_format
        if tools is not None:
            payload["tools"] = tools
    else:
        return "Invalid provider. Please choose from 'openai', 'anthropic', or 'mistral'."

    # Check if API key is available
    if not api_key:
        raise ValueError(f"{provider.capitalize()} API key not found. Please set it in the .env file.")
    
    try:
        # Send the POST request to the provider's API
        response = requests.post(url, headers=headers, json=payload)
        
        # Raise an exception if the request was unsuccessful
        response.raise_for_status()
        
        # Parse the JSON response
        response_data = response.json()
        
        # Extract the assistant's message and token information based on the provider
        input_tokens = len(" ".join(msg["content"] for msg in messages).split())  # Estimate of input tokens
        if provider.lower() == "openai":
            assistant_message = response_data["choices"][0]["message"]["content"].strip()
            output_tokens = response_data["usage"]["completion_tokens"]

        elif provider.lower() == "anthropic":
            assistant_message = response_data.get("content", [{}])[0].get("text", "No reply found.")
            output_tokens = len(assistant_message.split())  # Estimate of output tokens

        elif provider.lower() == "mistral":
            assistant_message = response_data.get("choices", [{}])[0].get("message", {}).get("content", "No reply found.")
            output_tokens = len(assistant_message.split())  # Estimate of output tokens
        
        # Format the response including provider, model, and token information
        formatted_text = (
            f"**Provider:** {provider.capitalize()} | **Model:** {model}  \n"
            f"**Tokens Used (Input/Output):** {input_tokens}/{output_tokens}  \n\n"
            f"**Assistant:**\n\n{assistant_message}\n"
        )
        return formatted_text

    except requests.exceptions.RequestException as e:
        return f"An error occurred: {e}"



In [2]:
# Define the conversation messages
messages = [
    {"role": "user", "content": "Explain the concept of quantum entanglement and how it challenges classical notions of locality and realism."}
]

# Generate and format the response from the specified provider
formatted_response = generate_and_format_response(
    provider="openai",        # Change to "anthropic" or "mistral" as needed
    model="gpt-3.5-turbo",    # Adjust the model for the chosen provider
    messages=messages,
    temperature=0.7,
    max_tokens=1500,
    top_p=0.9
)

# Display the formatted response
display(Markdown(formatted_response))

**Provider:** Openai | **Model:** gpt-3.5-turbo  
**Tokens Used (Input/Output):** 16/251  

**Assistant:**

Quantum entanglement is a phenomenon in quantum mechanics where two or more particles become interconnected in such a way that the state of one particle is dependent on the state of the other, regardless of the distance between them. This means that measuring the state of one particle instantaneously determines the state of the other, even if they are light-years apart.

This concept challenges classical notions of locality and realism because it violates the principle of locality, which states that an object is directly influenced only by its immediate surroundings. In the case of quantum entanglement, the state of one particle can be influenced by the state of another particle regardless of the distance between them, suggesting that there is some form of non-local connection between the particles.

Additionally, quantum entanglement challenges the concept of realism, which posits that objects have well-defined properties independent of observation. In the case of entangled particles, their properties are not well-defined until they are measured, and the act of measuring one particle instantly determines the properties of the other particle, even if they were previously unknown.

Overall, quantum entanglement challenges classical notions of locality and realism by demonstrating that particles can be interconnected in ways that defy our classical understanding of how objects interact with each other in the physical world.


In [3]:
# Define the conversation messages
messages = [
    {"role": "user", "content": "Explain the concept of quantum entanglement and how it challenges classical notions of locality and realism."}
]

# Generate and format the response for Anthropic
formatted_response = generate_and_format_response(
    provider="anthropic",                # Specify the provider as Anthropic
    model="claude-3-5-sonnet-20240620",  # Replace with your specific Anthropic model
    messages=messages,
    temperature=0.7,                     # Adjust for creativity
    max_tokens=1500,                     # Limit the length of the response
    top_p=0.9                            # Control diversity
)

# Display the formatted response
display(Markdown(formatted_response))

**Provider:** Anthropic | **Model:** claude-3-5-sonnet-20240620  
**Tokens Used (Input/Output):** 16/585  

**Assistant:**

Quantum entanglement is a fundamental concept in quantum mechanics that describes a phenomenon where two or more particles become intrinsically linked, such that the quantum state of each particle cannot be described independently of the others, even when separated by large distances. This concept challenges our classical understanding of physics and has profound implications for our understanding of reality, particularly in terms of locality and realism.

To understand how quantum entanglement challenges classical notions, let's break down the concept and its implications:

1. Quantum Entanglement Basics:

When particles become entangled, their quantum states are correlated in such a way that measuring the state of one particle instantaneously affects the state of its entangled partner(s), regardless of the distance between them. This correlation persists even when the particles are separated by vast distances, seemingly violating the principle of locality in classical physics.

2. Challenging Locality:

Locality in classical physics suggests that an object is influenced directly only by its immediate surroundings. The principle of locality states that an object can only be influenced by events in its immediate vicinity, and this influence cannot propagate faster than the speed of light.

Quantum entanglement appears to violate this principle because the measurement of one entangled particle seems to instantaneously affect its partner, even if they are separated by large distances. This "spooky action at a distance," as Einstein famously called it, suggests that information might be transmitted faster than the speed of light, which conflicts with special relativity.

3. Challenging Realism:

Classical realism assumes that physical properties of objects exist independently of observation. In other words, objects have definite properties whether we measure them or not.

Quantum entanglement challenges this notion because the properties of entangled particles are not determined until they are measured. Before measurement, the particles exist in a superposition of states, and it's only upon observation that their properties become defined. This suggests that reality might not be as objective or independent of observation as classical physics assumes.

4. The EPR Paradox and Bell's Theorem:

The Einstein-Podolsky-Rosen (EPR) paradox highlighted the apparent conflict between quantum mechanics and local realism. They argued that quantum mechanics must be incomplete because it allowed for "spooky action at a distance."

John Bell later developed Bell's Theorem, which provided a way to test whether quantum mechanics could be explained by local hidden variable theories (which attempt to preserve local realism). Subsequent experiments have consistently violated Bell's inequalities, supporting quantum mechanics and suggesting that local realism must be abandoned.

5. Implications and Interpretations:

The challenges posed by quantum entanglement to our classical understanding of physics have led to various interpretations of quantum mechanics, such as the Copenhagen interpretation, many-worlds interpretation, and quantum decoherence. These interpretations attempt to reconcile quantum phenomena with our understanding of reality, often leading to profound philosophical questions about the nature of existence and consciousness.

6. Practical Applications:

Despite its counterintuitive nature, quantum entanglement has practical applications in fields such as quantum computing, quantum cryptography, and quantum teleportation. These technologies exploit the unique properties of entangled particles to perform tasks that are impossible with classical systems.

In conclusion, quantum entanglement fundamentally challenges our classical notions of locality and realism by demonstrating non-local correlations and the dependence of reality on observation. This phenomenon forces us to reconsider our understanding of the nature of reality and the fundamental principles governing the universe. While it presents significant challenges to our intuitive understanding of physics, it also opens up new possibilities for technological advancements and deeper insights into the fabric of reality.


In [4]:
# Define the conversation messages
messages = [
    {"role": "user", "content": "Explain the concept of quantum entanglement and how it challenges classical notions of locality and realism. What are the implications of entanglement for our understanding of causality and information transfer?"}
]

# Generate and format the response for Mistral
formatted_response = generate_and_format_response(
    provider="mistral",                  # Specify the provider as Mistral
    model="mistral-small-latest",        # Replace with your specific Mistral model
    messages=messages,
    temperature=0.7,                     # Adjust for creativity
    max_tokens=2000,                     # Limit the length of the response
    top_p=0.9                            # Control diversity
)

# Display the formatted response
display(Markdown(formatted_response))


**Provider:** Mistral | **Model:** mistral-small-latest  
**Tokens Used (Input/Output):** 30/484  

**Assistant:**

Quantum entanglement is a phenomenon in quantum mechanics where two or more particles become correlated in such a way that the state of one particle cannot be described independently of the state of the other particles, even when the particles are separated by large distances. This correlation persists regardless of the distance between the particles, leading to what Einstein famously referred to as "spooky action at a distance."

### Challenges to Classical Notions

1. **Locality**: In classical physics, the idea of locality holds that an object is directly influenced only by its immediate surroundings. However, in quantum entanglement, the state of one particle can instantly affect the state of another, regardless of the distance between them. This instantaneous correlation violates the principle of locality as understood in classical physics.

2. **Realism**: Realism in physics assumes that particles have definite states independent of measurement. However, entanglement suggests that particles do not have definite states until measured. The act of measurement on one particle can instantly determine the state of another, which challenges the notion of realism.

### Implications for Causality and Information Transfer

1. **Causality**: Traditional notions of causality assume that effects follow their causes in a way that respects the speed of light. However, entanglement suggests that the state of one particle can instantaneously affect the state of another, seemingly violating the causal structure of spacetime. This is a profound challenge to our understanding of causality.

2. **Information Transfer**: Entanglement also challenges our understanding of information transfer. In classical physics, information cannot be transmitted faster than the speed of light. However, the correlations in entangled particles seem to allow for instantaneous information transfer, which is often referred to as "quantum teleportation." While this does not allow for the transmission of classical information faster than light (due to the no-communication theorem), it does suggest that information in a quantum system can be transferred in ways that are not possible in a classical system.

### Experimental Evidence and Theoretical Implications

- **Bell's Theorem**: John Bell formulated a set of inequalities (Bell's inequalities) that could be tested experimentally to distinguish between quantum mechanics and local hidden variable theories. Experiments have consistently violated these inequalities, supporting the predictions of quantum mechanics and challenging local realism.

- **Quantum Computing and Cryptography**: Entanglement is a key resource in quantum computing and quantum cryptography. Quantum computers use entangled qubits to perform calculations that are infeasible for classical computers. Quantum cryptography, such as quantum key distribution (QKD), uses entanglement to ensure secure communication.

### Conclusion

Quantum entanglement challenges our classical intuitions about locality and realism, and it has profound implications for our understanding of causality and information transfer. While it does not allow for faster-than-light communication, it does enable phenomena like quantum teleportation and quantum computing that are not possible in classical systems. The study of entanglement continues to be a vibrant area of research in quantum mechanics and quantum information theory.


In [6]:
import os
import requests
from dotenv import load_dotenv
from IPython.display import display, Markdown

# Load environment variables from .env
load_dotenv()

class ChatBot:
    def __init__(self, provider, model, api_key=None, temperature=0.7, max_tokens=1500, top_p=0.9, frequency_penalty=0.0, presence_penalty=0.0, min_tokens=None, stream=False, stop=None, random_seed=None, response_format=None, tools=None, tool_choice="auto", safe_prompt=False):
        """
        Initialize the ChatBot with API key, provider, and parameters.
        
        Parameters:
        - provider (str): The LLM provider to use ("openai", "anthropic", or "mistral").
        - model (str): The model to use for text generation.
        - api_key (str): The API key for the chosen provider.
        - temperature (float): Controls randomness in the output.
        - max_tokens (int): The maximum number of tokens to generate in the completion.
        - Additional parameters are specific to certain providers.
        """
        self.provider = provider.lower()
        self.model = model
        self.api_key = api_key or os.getenv(f"{self.provider.upper()}_API_KEY")
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.top_p = top_p
        self.frequency_penalty = frequency_penalty
        self.presence_penalty = presence_penalty
        self.min_tokens = min_tokens
        self.stream = stream
        self.stop = stop
        self.random_seed = random_seed
        self.response_format = response_format
        self.tools = tools
        self.tool_choice = tool_choice
        self.safe_prompt = safe_prompt
        self.conversation_history = []

    def add_message(self, role, content):
        """
        Add a message to the conversation history.
        
        Parameters:
        - role (str): The role of the sender ("user" or "assistant").
        - content (str): The content of the message.
        """
        self.conversation_history.append({"role": role, "content": content})

    def _call_api(self):
        """
        Internal method to call the appropriate API based on provider and generate a response.
        
        Returns:
        - response (dict): The API response as a dictionary.
        """
        headers = {"Content-Type": "application/json"}
        if self.provider == "openai":
            url = "https://api.openai.com/v1/chat/completions"
            headers["Authorization"] = f"Bearer {self.api_key}"
            payload = {
                "model": self.model,
                "messages": self.conversation_history,
                "temperature": self.temperature,
                "max_tokens": self.max_tokens,
                "top_p": self.top_p,
                "frequency_penalty": self.frequency_penalty,
                "presence_penalty": self.presence_penalty
            }

        elif self.provider == "anthropic":
            url = "https://api.anthropic.com/v1/messages"
            headers["x-api-key"] = self.api_key
            headers["anthropic-version"] = "2023-06-01"
            payload = {
                "model": self.model,
                "temperature": self.temperature,
                "max_tokens": self.max_tokens,
                "top_p": self.top_p,
                "messages": self.conversation_history
            }

        elif self.provider == "mistral":
            url = "https://api.mistral.ai/v1/chat/completions"
            headers["Authorization"] = f"Bearer {self.api_key}"
            payload = {
                "model": self.model,
                "messages": self.conversation_history,
                "temperature": self.temperature,
                "top_p": self.top_p,
                "stream": self.stream,
                "tool_choice": self.tool_choice,
                "safe_prompt": self.safe_prompt
            }
            # Add optional parameters for Mistral
            if self.max_tokens is not None:
                payload["max_tokens"] = self.max_tokens
            if self.min_tokens is not None:
                payload["min_tokens"] = self.min_tokens
            if self.stop is not None:
                payload["stop"] = self.stop
            if self.random_seed is not None:
                payload["random_seed"] = self.random_seed
            if self.response_format is not None:
                payload["response_format"] = self.response_format
            if self.tools is not None:
                payload["tools"] = self.tools
        else:
            raise ValueError("Invalid provider. Choose from 'openai', 'anthropic', or 'mistral'.")

        try:
            response = requests.post(url, headers=headers, json=payload)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"An error occurred: {e}")
            return None

    def get_response(self, user_input):
        """
        Get a response from the selected provider's API based on the user input.
        
        Parameters:
        - user_input (str): The user's input message.
        
        Returns:
        - formatted_response (str): The assistant's formatted response in Markdown.
        """
        # Add user input to conversation history
        self.add_message("user", user_input)

        # Call the API to get a response
        response = self._call_api()

        if response:
            # Extract the assistant's reply based on provider
            if self.provider == "openai":
                assistant_reply = response["choices"][0]["message"]["content"]
                token_usage = response["usage"]["completion_tokens"]

            elif self.provider == "anthropic":
                assistant_reply = response.get("content", [{}])[0].get("text", "No reply found.")
                token_usage = len(assistant_reply.split())  # Estimated tokens for Anthropic

            elif self.provider == "mistral":
                assistant_reply = response.get("choices", [{}])[0].get("message", {}).get("content", "No reply found.")
                token_usage = len(assistant_reply.split())  # Estimated tokens for Mistral

            # Add assistant's reply to conversation history
            self.add_message("assistant", assistant_reply)

            # Format the response using Markdown
            formatted_response = (
                f"**Provider:** {self.provider.capitalize()} | **Model:** {self.model}  \n"
                f"**Tokens Used:** {token_usage}  \n\n"
                f"**Assistant:**\n\n{assistant_reply}\n"
            )
            return formatted_response
        else:
            return "Sorry, I couldn't generate a response."




In [7]:
# Example Usage
# Initialize ChatBot with OpenAI, Anthropic, or Mistral

# Replace 'openai' with 'anthropic' or 'mistral' as needed
bot = ChatBot(provider="openai", model="gpt-4o", temperature=1.0, max_tokens=2000, top_p=1.0)

# Get a response from the chatbot
user_input = "Can you suggest 5 dinner ideas for this week?"
response = bot.get_response(user_input)

# Display the formatted response as Markdown
display(Markdown(response))

**Provider:** Openai | **Model:** gpt-4o  
**Tokens Used:** 326  

**Assistant:**

Certainly! Here are five dinner ideas for you to consider this week:

1. **Monday: Lemon Herb Grilled Chicken with Quinoa Salad**
   - Marinate chicken breasts in olive oil, lemon juice, garlic, and herbs like thyme and rosemary. Grill until cooked through.
   - Serve with a quinoa salad tossed with cherry tomatoes, cucumber, red onion, feta cheese, and a lemon vinaigrette.

2. **Tuesday: Spaghetti Aglio e Olio**
   - Cook spaghetti and toss it with sautéed garlic in olive oil, red pepper flakes, and parsley.
   - Serve with freshly grated parmesan and a side of garlic bread.

3. **Wednesday: Beef Stir-Fry with Broccoli and Bell Peppers**
   - Sauté sliced beef with garlic, ginger, and a mix of soy sauce and sesame oil.
   - Add broccoli and bell peppers and stir-fry until tender. Serve over steamed rice.

4. **Thursday: Baked Salmon with Roasted Vegetables**
   - Season salmon fillets with salt, pepper, and dill. Bake in the oven until just cooked.
   - Pair with a mix of roasted potatoes, carrots, and Brussel sprouts, drizzled with olive oil and seasoned with rosemary.

5. **Friday: Vegetarian Tacos with Black Beans and Avocado**
   - Fill corn tortillas with a mixture of seasoned black beans, diced avocado, and chopped tomatoes.
   - Top with shredded lettuce, cheese, sour cream, and freshly squeezed lime juice.

These meals are not only easy to prepare but also nutritious and full of flavor. Enjoy your dinners!


In [8]:
# Initialize ChatBot with Anthropic
bot = ChatBot(
    provider="anthropic", 
    model="claude-3-5-sonnet-20240620",  # Replace with your specific Anthropic model if needed
    temperature=0.7, 
    max_tokens=1500, 
    top_p=0.9
)

# Get a response from the chatbot
user_input = "Explain the concept of quantum entanglement and its implications for understanding causality."
response = bot.get_response(user_input)

# Display the formatted response as Markdown
display(Markdown(response))

**Provider:** Anthropic | **Model:** claude-3-5-sonnet-20240620  
**Tokens Used:** 614  

**Assistant:**

Quantum entanglement is a fascinating phenomenon in quantum mechanics that has profound implications for our understanding of the nature of reality, causality, and information. To explain this concept and its implications, let's break it down into several key points:

1. Definition of Quantum Entanglement:
Quantum entanglement occurs when two or more particles become correlated in such a way that the quantum state of each particle cannot be described independently of the others, even when the particles are separated by a large distance. In other words, the particles become intrinsically linked, and their properties are fundamentally connected.

2. Einstein-Podolsky-Rosen (EPR) Paradox:
The concept of quantum entanglement was first highlighted by Einstein, Podolsky, and Rosen in 1935 as a challenge to the completeness of quantum mechanics. They argued that entanglement implied the existence of "spooky action at a distance," which seemed to violate the principle of locality in classical physics.

3. Bell's Theorem:
In 1964, John Stewart Bell proposed a way to test whether quantum entanglement could be explained by local hidden variables. His theorem showed that the correlations predicted by quantum mechanics are stronger than what could be explained by any local hidden variable theory, leading to experimental tests that have consistently supported quantum mechanics.

4. Experimental Confirmation:
Numerous experiments, including those by Alain Aspect in the 1980s and more recent ones, have demonstrated the reality of quantum entanglement, confirming that entangled particles can indeed influence each other instantaneously, regardless of their separation in space.

Implications for Causality:

1. Non-locality:
Quantum entanglement challenges our classical notion of locality, which states that objects can only be influenced by their immediate surroundings. Entanglement suggests that there can be instantaneous correlations between distant particles, seemingly violating the speed of light limit for information transfer.

2. Faster-than-light Communication?:
While entanglement allows for instantaneous correlations, it's important to note that it cannot be used for faster-than-light communication. The outcomes of measurements on entangled particles are random, and no information can be transmitted intentionally through this process.

3. Quantum Causality:
Entanglement forces us to reconsider our understanding of causality. In the quantum world, cause and effect relationships become blurred, and the idea of a definite sequence of events becomes less clear-cut.

4. Quantum Information:
Entanglement is a crucial resource in quantum information theory and quantum computing. It allows for phenomena like quantum teleportation and super-dense coding, which have no classical analogues.

5. Holistic Nature of Reality:
Entanglement suggests that the universe may be fundamentally interconnected at a quantum level, challenging reductionist views of reality and pointing towards a more holistic understanding of nature.

6. Time and Entanglement:
Some interpretations of quantum mechanics, such as the transactional interpretation, suggest that entanglement might involve connections not just across space but also across time, further complicating our understanding of causality.

7. Measurement Problem:
Entanglement is closely related to the measurement problem in quantum mechanics. The act of measuring one entangled particle instantaneously affects the state of its partner, raising questions about the nature of measurement and the role of consciousness in quantum phenomena.

8. Quantum Foundations:
The phenomenon of entanglement continues to be a central topic in discussions about the foundations of quantum mechanics and the nature of reality itself. It has led to various interpretations of quantum mechanics, each with different implications for causality and the nature of the quantum world.

In conclusion, quantum entanglement challenges our classical notions of causality, locality, and the nature of information. It suggests a universe that is more interconnected and less deterministic than classical physics would have us believe. While the full implications of entanglement are still being explored, it has already revolutionized our understanding of the quantum world and opened up new possibilities in quantum technologies.


In [9]:
# Initialize ChatBot with Mistral
bot = ChatBot(
    provider="mistral", 
    model="mistral-small-latest",  # Replace with your specific Mistral model if needed
    temperature=0.7, 
    max_tokens=1500, 
    top_p=0.9
)

# Get a response from the chatbot
user_input = "Explain the concept of quantum entanglement and its implications for understanding causality."
response = bot.get_response(user_input)

# Display the formatted response as Markdown
display(Markdown(response))

**Provider:** Mistral | **Model:** mistral-small-latest  
**Tokens Used:** 455  

**Assistant:**

Quantum entanglement is a phenomenon in quantum physics where two or more particles become correlated in such a way that the state of one particle cannot be described independently of the state of the other particles, even when they are separated by large distances. This correlation persists regardless of the distance between the particles, which led Albert Einstein to famously refer to it as "spooky action at a distance."

### Key Concepts of Quantum Entanglement:

1. **Superposition**: Quantum particles can exist in multiple states simultaneously until they are measured. This is known as superposition.

2. **Correlation**: When particles become entangled, the state of one particle is instantly correlated with the state of the other(s). For example, if you measure an entangled particle and find it in a specific state (e.g., spin up), the other entangled particle will immediately be in the corresponding state (e.g., spin down), no matter the distance between them.

3. **Measurement**: The act of measuring one particle's state instantaneously determines the state of the other particle, even if they are light-years apart.

### Implications for Understanding Causality:

1. **Non-locality**: Quantum entanglement challenges our classical understanding of causality, which assumes that an effect cannot precede its cause and that information cannot travel faster than the speed of light. The instantaneous correlation between entangled particles suggests that there might be a form of instantaneous action at a distance, which is non-local.

2. **Bell's Theorem**: John Bell formulated inequalities (Bell's inequalities) that could be tested experimentally to distinguish between quantum mechanics and classical theories. Experimental results consistently support quantum mechanics, suggesting that local hidden variables cannot explain the correlations observed in entangled systems.

3. **Einstein-Podolsky-Rosen (EPR) Paradox**: The EPR paradox highlighted the tension between quantum mechanics and the principle of locality. Entanglement implies that either the principle of locality is violated, or there are hidden variables that we do not yet understand.

4. **Quantum Information Theory**: Entanglement is a crucial resource in quantum information science, enabling phenomena like quantum teleportation and quantum computing. It suggests that information can be processed and transmitted in ways that are fundamentally different from classical systems.

5. **Foundations of Quantum Mechanics**: The implications of entanglement for causality are still a subject of debate and research. Some interpretations of quantum mechanics, such as the many-worlds interpretation and the transactional interpretation, offer different perspectives on how to reconcile entanglement with our intuitive notions of causality.

### Conclusion:

Quantum entanglement has profound implications for our understanding of causality. It challenges the classical notion of locality and suggests that there might be deeper, non-local aspects of reality that we do not yet fully comprehend. Ongoing research aims to further elucidate these phenomena and potentially integrate them into a more comprehensive understanding of the universe.


In [11]:
import os
import requests
from dotenv import load_dotenv
from IPython.display import display, Markdown

# Load environment variables from .env
load_dotenv()

def retrieve_relevant_docs(query, documents):
    """
    Simple keyword-based function to retrieve relevant documents.
    """
    relevant_docs = []
    for doc in documents:
        if any(keyword.lower() in doc.lower() for keyword in query.split()):
            relevant_docs.append(doc)
    return relevant_docs

def generate_and_format_response(
    provider,
    model,
    messages,
    temperature=0.7,
    max_tokens=1500,
    top_p=0.9,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    min_tokens=None,
    stream=False,
    stop=None,
    random_seed=None,
    response_format=None,
    tools=None,
    tool_choice="auto",
    safe_prompt=False
):
    """
    Generates and formats a response from the specified LLM provider (OpenAI, Anthropic, or Mistral) in Markdown format.
    """
    api_key = None
    url = None
    headers = {"Content-Type": "application/json"}

    # Determine API key, endpoint, and headers based on provider
    if provider.lower() == "openai":
        api_key = os.getenv("OPENAI_API_KEY")
        url = "https://api.openai.com/v1/chat/completions"
        headers["Authorization"] = f"Bearer {api_key}"
        payload = {
            "model": model,
            "messages": messages,
            "temperature": temperature,
            "max_tokens": max_tokens,
            "top_p": top_p,
            "frequency_penalty": frequency_penalty,
            "presence_penalty": presence_penalty
        }

    elif provider.lower() == "anthropic":
        api_key = os.getenv("ANTHROPIC_API_KEY")
        url = "https://api.anthropic.com/v1/messages"
        headers["x-api-key"] = api_key
        headers["anthropic-version"] = "2023-06-01"
        payload = {
            "model": model,
            "temperature": temperature,
            "max_tokens": max_tokens,
            "top_p": top_p,
            "messages": messages
        }

    elif provider.lower() == "mistral":
        api_key = os.getenv("MISTRAL_API_KEY")
        url = "https://api.mistral.ai/v1/chat/completions"
        headers["Authorization"] = f"Bearer {api_key}"
        payload = {
            "model": model,
            "messages": messages,
            "temperature": temperature,
            "top_p": top_p,
            "stream": stream,
            "tool_choice": tool_choice,
            "safe_prompt": safe_prompt
        }
        if max_tokens is not None:
            payload["max_tokens"] = max_tokens
        if min_tokens is not None:
            payload["min_tokens"] = min_tokens
        if stop is not None:
            payload["stop"] = stop
        if random_seed is not None:
            payload["random_seed"] = random_seed
        if response_format is not None:
            payload["response_format"] = response_format
        if tools is not None:
            payload["tools"] = tools
    else:
        return "Invalid provider. Please choose from 'openai', 'anthropic', or 'mistral'."

    if not api_key:
        raise ValueError(f"{provider.capitalize()} API key not found. Please set it in the .env file.")
    
    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()
        response_data = response.json()
        
        input_tokens = len(" ".join(msg["content"] for msg in messages).split())
        if provider.lower() == "openai":
            assistant_message = response_data["choices"][0]["message"]["content"].strip()
            output_tokens = response_data["usage"]["completion_tokens"]

        elif provider.lower() == "anthropic":
            assistant_message = response_data.get("content", [{}])[0].get("text", "No reply found.")
            output_tokens = len(assistant_message.split())

        elif provider.lower() == "mistral":
            assistant_message = response_data.get("choices", [{}])[0].get("message", {}).get("content", "No reply found.")
            output_tokens = len(assistant_message.split())
        
        formatted_text = (
            f"**Provider:** {provider.capitalize()} | **Model:** {model}  \n"
            f"**Tokens Used (Input/Output):** {input_tokens}/{output_tokens}  \n\n"
            f"**Assistant:**\n\n{assistant_message}\n"
        )
        return formatted_text

    except requests.exceptions.RequestException as e:
        return f"An error occurred: {e}"

def generate_response_with_rag(
    provider,
    model,
    conversation_history,
    current_message,
    documents,
    temperature=0.7,
    max_tokens=1500,
    top_p=0.9,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    min_tokens=None,
    stream=False,
    stop=None,
    random_seed=None,
    response_format=None,
    tools=None,
    tool_choice="auto",
    safe_prompt=False
):
    """
    Wrapper function to perform RAG (Retrieval-Augmented Generation) by retrieving relevant documents,
    combining them into a context, and generating a response with the specified provider.
    """
    relevant_docs = retrieve_relevant_docs(current_message, documents)
    context = "\n\n".join(relevant_docs)
    augmented_message = f"Context: {context}\n\n{current_message}"
    messages = conversation_history + [{"role": "user", "content": augmented_message}]

    response = generate_and_format_response(
        provider=provider,
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty,
        min_tokens=min_tokens,
        stream=stream,
        stop=stop,
        random_seed=random_seed,
        response_format=response_format,
        tools=tools,
        tool_choice=tool_choice,
        safe_prompt=safe_prompt
    )

    # Display formatted response as Markdown
    display(Markdown(response))
    return response


In [12]:
# Define conversation history and current message
conversation_history = [
    {"role": "assistant", "content": "Hello! How can I assist you today?"}
]
current_message = "What is the debt-to-equity ratio of ABC Corp?"

# Sample documents for retrieval
documents = [
    "ABC Corp. reported a revenue of 50 million for Q2 2024, a 10 percent increase from Q1 2024.",
    "ABC Corp. has a current debt-to-equity ratio of 0.3, indicating that the company has a low level of debt compared to its equity.",
    "The market capitalization of ABC Corp. is currently 300 million.",
    "In Q2 2024, ABC Corp. announced a dividend of 0.50 per share.",
    "ABC Corp.'s gross profit margin for Q2 2024 was 40 percent."
]

# Generate response with RAG using Mistral as the provider
formatted_response = generate_response_with_rag(
    provider="mistral", 
    model="mistral-small-latest",  # Replace with your specific Mistral model
    conversation_history=conversation_history, 
    current_message=current_message, 
    documents=documents, 
    temperature=0.7, 
    max_tokens=256, 
    top_p=0.9
)


**Provider:** Mistral | **Model:** mistral-small-latest  
**Tokens Used (Input/Output):** 90/28  

**Assistant:**

The debt-to-equity ratio of ABC Corp. is given as 0.3 in the context provided. This means that for every dollar of equity, the company has $0.30 of debt.


In [13]:
# Define conversation history and current message
conversation_history = [
    {"role": "assistant", "content": "Hello! How can I assist you today?"}
]
current_message = "What is the debt-to-equity ratio of ABC Corp?"

# Sample documents for retrieval
documents = [
    "ABC Corp. reported a revenue of 50 million for Q2 2024, a 10 percent increase from Q1 2024.",
    "ABC Corp. has a current debt-to-equity ratio of 0.3, indicating that the company has a low level of debt compared to its equity.",
    "The market capitalization of ABC Corp. is currently 300 million.",
    "In Q2 2024, ABC Corp. announced a dividend of 0.50 per share.",
    "ABC Corp.'s gross profit margin for Q2 2024 was 40 percent."
]

# Generate response with RAG using OpenAI as the provider
formatted_response = generate_response_with_rag(
    provider="openai", 
    model="gpt-4",  # Replace with your specific OpenAI model
    conversation_history=conversation_history, 
    current_message=current_message, 
    documents=documents, 
    temperature=0.7, 
    max_tokens=256, 
    top_p=0.9
)

# The formatted response is displayed directly in Markdown

**Provider:** Openai | **Model:** gpt-4  
**Tokens Used (Input/Output):** 90/16  

**Assistant:**

The debt-to-equity ratio of ABC Corp. is 0.3.


In [14]:
# Define conversation history and current message
conversation_history = [
    {"role": "assistant", "content": "Hello! How can I assist you today?"}
]
current_message = "What is the debt-to-equity ratio of ABC Corp?"

# Sample documents for retrieval
documents = [
    "ABC Corp. reported a revenue of 50 million for Q2 2024, a 10 percent increase from Q1 2024.",
    "ABC Corp. has a current debt-to-equity ratio of 0.3, indicating that the company has a low level of debt compared to its equity.",
    "The market capitalization of ABC Corp. is currently 300 million.",
    "In Q2 2024, ABC Corp. announced a dividend of 0.50 per share.",
    "ABC Corp.'s gross profit margin for Q2 2024 was 40 percent."
]

# Generate response with RAG using Anthropic as the provider
formatted_response = generate_response_with_rag(
    provider="anthropic", 
    model="claude-3-5-sonnet-20240620",  # Replace with your specific Anthropic model
    conversation_history=conversation_history, 
    current_message=current_message, 
    documents=documents, 
    temperature=0.7, 
    max_tokens=256, 
    top_p=0.9
)

**Provider:** Anthropic | **Model:** claude-3-5-sonnet-20240620  
**Tokens Used (Input/Output):** 90/93  

**Assistant:**

Based on the information provided in the context, the debt-to-equity ratio of ABC Corp. is 0.3.

The debt-to-equity ratio is a financial metric that compares a company's total debt to its total shareholders' equity. It's used to evaluate a company's financial leverage and risk. A lower ratio generally indicates that a company is using less debt financing relative to equity financing.

In this case, ABC Corp.'s debt-to-equity ratio of 0.3 is considered relatively low, suggesting that the company has a conservative capital structure with a lower level of debt compared to its equity.


In [31]:
import os
import requests
import pdfplumber
from dotenv import load_dotenv
from IPython.display import display, Markdown

# Load environment variables from .env
load_dotenv()

# Maximum token limit for chunk processing
MAX_TOKENS = 8192 // 2  # Divide to ensure each chunk is within model limits

def pdf_to_text(pdf_path):
    """
    Extracts text from a PDF file.
    """
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

def split_text_into_chunks(text, max_tokens):
    """
    Splits text into chunks to fit within the token limit.
    """
    words = text.split()
    chunks = []
    current_chunk = []
    current_length = 0
    for word in words:
        current_chunk.append(word)
        current_length += len(word) + 1  # +1 for space
        if current_length >= max_tokens:
            chunks.append(" ".join(current_chunk))
            current_chunk = []
            current_length = 0
    if current_chunk:
        chunks.append(" ".join(current_chunk))
    return chunks

def summarize_chunk(provider, api_key, chunk, model="gpt-4o-mini", temperature=0.7, max_tokens=200):
    """
    Summarizes a single chunk of text using the specified provider.
    """
    prompt = f"Summarize the following text in one sentence:\n\n{chunk}"
    messages = [{"role": "user", "content": prompt}]
    
    return generate_and_format_response(
        provider=provider,
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )

def summarize_pdf_with_provider(provider, pdf_path, model="gpt-4o-mini", temperature=0.7, max_tokens=200):
    """
    Summarize chunks privide.
    """
    # Convert PDF to text
    text = pdf_to_text(pdf_path)

    # Split text into chunks
    chunks = split_text_into_chunks(text, MAX_TOKENS)

    # Summarize each chunk
    summaries = []
    for i, chunk in enumerate(chunks):
        print(f"Processing chunk {i+1}/{len(chunks)}")
        summary = summarize_chunk(provider, os.getenv(f"{provider.upper()}_API_KEY"), chunk, model=model, temperature=temperature, max_tokens=max_tokens)
        summaries.append(summary)
        print(f"Chunk {i+1} summarized.")

    # Combine chunk summaries to generate final summary
    combined_summary = " ".join(summaries)
    final_summary_prompt = f"Summarize the following text in a concise summary:\n\n{combined_summary}"
    final_summary = generate_and_format_response(
        provider=provider,
        model=model,
        messages=[{"role": "user", "content": final_summary_prompt}],
        temperature=temperature,
        max_tokens=max_tokens
    )

    # Display the final summary in Markdown format
    display(Markdown(final_summary))


In [32]:
# Example usage
pdf_path = "sample_document.pdf"  # Replace with the path to your PDF
summarize_pdf_with_provider(provider="anthropic", pdf_path=pdf_path, model="claude-3-5-sonnet-20240620", temperature=0.7, max_tokens=512)


Processing chunk 1/7
Chunk 1 summarized.
Processing chunk 2/7
Chunk 2 summarized.
Processing chunk 3/7
Chunk 3 summarized.
Processing chunk 4/7
Chunk 4 summarized.
Processing chunk 5/7
Chunk 5 summarized.
Processing chunk 6/7
Chunk 6 summarized.
Processing chunk 7/7
Chunk 7 summarized.


**Provider:** Anthropic | **Model:** claude-3-5-sonnet-20240620  
**Tokens Used (Input/Output):** 359/104  

**Assistant:**

Here is a concise summary of the key points from the provided text excerpts:

The Bank of Canada's restrictive monetary policy has successfully reduced inflation but may now be too aggressive, potentially harming economic growth and employment. Canada is facing labor market challenges including declining employment and weak job creation. Interest rate cuts are expected due to lower inflation pressures and a cooling labor market. The text also includes economic forecasts for Canadian provinces/territories and national indicators. Legal disclaimers emphasize the report is not independent research or investment advice. Detailed regulatory information is provided about National Bank of Canada's financial services in various jurisdictions.


In [33]:
# Example usage
pdf_path = "sample_document.pdf"  # Replace with the path to your PDF
summarize_pdf_with_provider(provider="openai", pdf_path=pdf_path, model="gpt-4o-mini", temperature=0.7, max_tokens=512)


Processing chunk 1/7
Chunk 1 summarized.
Processing chunk 2/7
Chunk 2 summarized.
Processing chunk 3/7
Chunk 3 summarized.
Processing chunk 4/7
Chunk 4 summarized.
Processing chunk 5/7
Chunk 5 summarized.
Processing chunk 6/7
Chunk 6 summarized.
Processing chunk 7/7
Chunk 7 summarized.


**Provider:** Openai | **Model:** gpt-4o-mini  
**Tokens Used (Input/Output):** 352/83  

**Assistant:**

The text discusses the need for the Bank of Canada to reconsider its restrictive monetary policy due to high interest rates negatively impacting economic growth and the labor market, despite lower inflation. It highlights stagnation in the Canadian labor market, differing inflationary pressures between the US and Canada, and provides various economic forecasts and financial data from National Bank Financial, emphasizing the document's nature as a marketing tool for professional investors without regulatory oversight.


In [34]:
# Example usage
pdf_path = "sample_document.pdf"  # Replace with the path to your PDF
summarize_pdf_with_provider(provider="mistral", pdf_path=pdf_path, model="mistral-small-latest", temperature=0.7, max_tokens=512)


Processing chunk 1/7
Chunk 1 summarized.
Processing chunk 2/7
Chunk 2 summarized.
Processing chunk 3/7
Chunk 3 summarized.
Processing chunk 4/7
Chunk 4 summarized.
Processing chunk 5/7
Chunk 5 summarized.
Processing chunk 6/7
Chunk 6 summarized.
Processing chunk 7/7
Chunk 7 summarized.


**Provider:** Mistral | **Model:** mistral-small-latest  
**Tokens Used (Input/Output):** 291/68  

**Assistant:**

The text discusses the Bank of Canada's monetary policy, recommending easing due to controlled inflation but weak economic growth and labor market. It also highlights Canada's labor market struggles, divergent monetary policies with the US, and varying inflation pressures. Economic forecasts for Canada from 2021 to 2025 are provided, along with a marketing document disclaimer from National Bank Financial Inc. regarding its investment services and report distribution restrictions.
